# Adversarial Dropout Code
#### This file contains the code used to replicate the results of Adversarial Dropout

In [0]:
import json
import re
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.datasets import mnist
from matplotlib import pyplot as plt
from google.colab import files

## Layers

In [0]:
SEED = 123456
rng = np.random.RandomState(SEED)


def __createWeights(shape, seed=None, name='weight'):
    w_init = tf.contrib.layers.variance_scaling_initializer(seed=seed)
    return tf.get_variable(name + '_w', shape=shape, initializer=w_init)
    

def __createBiases(size, name='bias'):
    return tf.get_variable(name + '_b', shape=[size], initializer=tf.constant_initializer(0.0))


def LeakyReLU(x, alpha=0.1):
    x = tf.nn.leaky_relu(x, alpha=alpha)
    return x


def MaxPooling(x, ksize=2, stride_length=2, padding='SAME', data_format='NHWC'):
    x = tf.nn.max_pool(x, (1, ksize, ksize, 1), (1, stride_length, stride_length, 1), padding, data_format)
    return x


def GlobalAveragePooling(x):
    x = tf.reduce_mean(x, [1, 2])
    return x


def Dense(x, input_dim, output_dim, seed=None, name='dense'):
    W = __createWeights([input_dim, output_dim], seed, name) 
    b = __createBiases(output_dim, name) 
    x = tf.nn.xw_plus_b(x, W, b)
    return x


def Conv2D(x, filter_size, n_channels, n_filters, stride_length=1, padding='SAME', data_format='NHWC', name='conv'):
    shape = [filter_size, filter_size, n_channels, n_filters]
    W = __createWeights(shape, name=name)
    b = __createBiases(n_filters, name=name)
    x = tf.nn.conv2d(x, filter=W, strides=(1, stride_length, stride_length, 1), padding=padding, data_format=data_format)
    x += b
    return x


def Dropout(x, rate=0.5, is_train=True):
    x = tf.layers.dropout(x, rate=rate, seed=rng.randint(SEED), training=is_train)
    return x


def GaussianNoise(x, sigma=0.15):
    noise = tf.random_normal(shape=tf.shape(x), stddev=sigma)
    x += noise
    return x


def SoftMax(x):
    x = tf.nn.softmax(x)
    return x


def Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='bn'):
    x = tf.layers.batch_normalization(x, axis=axis, training=is_train, momentum=momentum, name=name)
    return x



'''
Loss functions. Arg 1: Approximation, Arg 2: Labels
'''
def CrossEntropyWithLogits(logits, labels):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss


# Formula: sum(p_i * log(p_i) - p_i * log(q_i))
def KLDivergenceWithLogits(q, p):
    p_soft = SoftMax(p)
    # plogp = tf.reduce_mean(tf.reduce_sum(p_soft * tf.nn.log_softmax(p), 1))
    # plogq = tf.reduce_mean(tf.reduce_sum(p_soft * tf.nn.log_softmax(q), 1))
    distance = tf.reduce_sum(p_soft * tf.nn.log_softmax(p) - p_soft * tf.nn.log_softmax(q))
    # distance = plogp - plogq
    return distance
 
    

## Building blocks for creating the networks

In [0]:
'''
The model before AdD is applied
'''
def upperBlock(x, conv_size=[128, 256, 512, 256, 128], n_channels=3, is_train=True):
    x = GaussianNoise(x)
    x = Conv2D(x, filter_size=3, n_channels=n_channels, n_filters=conv_size[0], padding='SAME', name='1a')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='1bna')
    x = LeakyReLU(x)
    x = Conv2D(x, filter_size=3, n_channels=conv_size[0], n_filters=conv_size[0], name='1b')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='1bnb')
    x = LeakyReLU(x)
    x = Conv2D(x, filter_size=3, n_channels=conv_size[0], n_filters=conv_size[0], name='1c')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='1bnc')
    x = MaxPooling(x, ksize=2, stride_length=2)
    x = Dropout(x, 0.5, is_train)
    
    x = Conv2D(x, filter_size=3, n_channels=conv_size[0], n_filters=conv_size[1], name='2a')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='2bna')
    x = LeakyReLU(x)
    x = Conv2D(x, filter_size=3, n_channels=conv_size[1], n_filters=conv_size[1], name='2b')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='2bnb')
    x = LeakyReLU(x)
    x = Conv2D(x, filter_size=3, n_channels=conv_size[1], n_filters=conv_size[1], name='2c')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='2bnc')
    x = LeakyReLU(x)
    x = MaxPooling(x, ksize=2, stride_length=2)
    x = Dropout(x, 0.5, is_train)

    x = Conv2D(x, filter_size=3, n_channels=conv_size[1], n_filters=conv_size[2], padding='VALID', name='3a')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='3bna')
    x = LeakyReLU(x)
    x = Conv2D(x, filter_size=1, n_channels=conv_size[2], n_filters=conv_size[3], name='3b')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='3bnb')
    x = LeakyReLU(x)
    x = Conv2D(x, filter_size=1, n_channels=conv_size[3], n_filters=conv_size[4], name='3c')
    x = Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='3bnc')
    x = LeakyReLU(x)
    x = GlobalAveragePooling(x) 

    return x


'''
The model after AdD is applied
'''
def lowerBlock(x, n_in=128, n_out=10, name='fc'):
    x = Dense(x, n_in, n_out, name=name)
    return x;


'''
Apply adv dropout
'''
def advDropout(x, mask, Jacobian, sigma=0.05, dim=128):
    # y: output 
    # mask: current sampled dropout mask 
    # sigma: hyper-parameter for boundary 
    # Jabocian: Jacobian vector (gradient of divergence (or loss function))
    # dim: layer dimension 

    Jacobian = tf.reshape(Jacobian, [-1, dim])

    # mask = 0 --> -1 
    mask = 2 * mask - tf.ones_like(mask)

    adv_mask = mask 

    # extract the voxels for which the update conditions hold 
    # mask = 0 and J > 0 
    # or
    # mask = 1 and J < 1 
    abs_jac = tf.abs(Jacobian)
    temp = tf.cast(tf.greater(abs_jac, 0), tf.float32)
    temp = 2 * temp - 1 
    # interested in the cases when temp * mask = -1
    ext = tf.cast(tf.less(mask, temp), tf.float32)

    # keep the voxels that you want to update 
    candidates = abs_jac * ext 
    thres = tf.nn.top_k(candidates, int(dim * sigma * sigma)  + 1)[0][:,-1]

    targets = tf.cast(tf.greater(candidates, tf.expand_dims(thres, -1)), tf.float32)

    # get new mask 
    adv_mask = (mask - targets * 2 * mask + tf.ones_like(mask)) / 2.0

    output = adv_mask * x

    return output, adv_mask

## Models

In [0]:
''' Preprocesses the data '''
def preprocess(data):
    # Mean normalization
    data = (data - 127.5) / 255.
    # Find principal component
    shape = data.shape
    data = data.transpose(0, 2, 3, 1)
    flatx = np.reshape(data, (data.shape[0], data.shape[1] * data.shape[2] * data.shape[3]))
    sigma = np.dot(flatx.T, flatx) / flatx.shape[1]
    U, S, V = np.linalg.svd(sigma)
    pc = np.dot(np.dot(U, np.diag(1. / np.sqrt(S + 0.0001))), U.T)
    # Apply ZCA whitening
    whitex = np.dot(flatx, pc)
    data = np.reshape(whitex, (shape[0], shape[1], shape[2], shape[3]))
    return data


'''
Returns a model without adversarial dropout
'''
def modelWithRandD(x, n_channels=3, is_train=True):
    x = upperBlock(x, n_channels=n_channels, is_train=is_train)
    x = lowerBlock(x)
    return x


'''
Returns a model with adversarial dropout
'''
def modelWithAdD(x, y, fn_loss=KLDivergenceWithLogits, n_channels=3, is_train=True):
    x = upperBlock(x, n_channels=n_channels, is_train=is_train)
    y_no_adD = lowerBlock(x)
    loss_no_adD = fn_loss(y_no_adD, y)

    # Derivative of loss fn wrt x
    DLoss = tf.gradients(loss_no_adD, [x])
    DLoss = tf.squeeze(tf.stop_gradient(DLoss)) # Stops backpropagation

    Jacobian_approx = DLoss * x
    mask = tf.ones_like(x)

    x, _ = advDropout(x, mask, Jacobian_approx)
    x = lowerBlock(x)

    return x



def CreateBaseModel(x, y, learning_rate=0.001, optimizer=tf.train.AdamOptimizer, n_channels=3, is_train=True):
    logit_rand = modelWithRandD(x, n_channels=n_channels)
    loss = CrossEntropyWithLogits(logit_rand, y)

    opt = optimizer(learning_rate=learning_rate)
    gradients = opt.compute_gradients(loss, tf.trainable_variables())
    train_op = opt.apply_gradients(gradients)

    return train_op, loss, logit_rand


'''
Create the AdD model for training
'''
def CreateAdDModel(x, y, learning_rate=0.001, optimizer=tf.train.AdamOptimizer, lmb=0.01, n_channels=3, is_train=True):
    logit_rand = modelWithRandD(x, n_channels=n_channels, is_train=is_train)
    logit_rand_loss = CrossEntropyWithLogits(logit_rand, y)

    with tf.variable_scope(tf.get_variable_scope(), reuse=True) as scope:
        # With adversarial dropout
        logit_adD = modelWithAdD(x, y, n_channels=n_channels, is_train=is_train)
        logit_adD_loss = CrossEntropyWithLogits(logit_adD, y)

        # Total loss
        loss = logit_rand_loss + lmb * logit_adD_loss

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt = optimizer(learning_rate=learning_rate)
        gradients = opt.compute_gradients(loss, tf.trainable_variables())
        train_op = opt.apply_gradients(gradients)        

    return train_op, loss, logit_rand


def CreateTestModel(x, y, n_channels=3, is_train=True):  
    with tf.variable_scope(tf.get_variable_scope(), reuse=True) as scope:         
        logit_rand = modelWithRandD(x, n_channels=n_channels, is_train=is_train)
        logit_rand_loss = CrossEntropyWithLogits(logit_rand, y)

        return logit_rand_loss, logit_rand


def Accuracy(logits, labels):
    y_pred = tf.argmax(logits, 1)
    y_true = tf.argmax(labels, 1)
    equality = tf.equal(y_pred, y_true)
    accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
    return accuracy


#### Visualization functions

In [0]:
'''
Uncomment to download graphs from google colab
'''
from google.colab import files

def visualize(trend, param):
    name = "Baseline" if param['BASELINE'] else "Adversarial"
    
    for key, val in trend.items():
        # X axis
        x = np.arange(len(val))
        # Acc or loss
        title = re.sub("_", " ", key).capitalize()
        plt.title(title + " " + name)
        plt.plot(x, val)
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy' if 'acc' in key else 'Loss')
        filename = str(key + re.sub("{|}|:|'|,| ", "", str(param)) + ".png")
        print(filename)
        plt.savefig(filename)
        files.download(filename)
        plt.close()
        
        
def visualize_both(trend_base, trend_adv, param_base, param_adv):
    for key in trend_base.keys():
        # X axis
        x = np.arange(min(len(trend_base[key]), len(trend_adv[key])))
        # Acc or loss
        title = re.sub("_", " ", key).capitalize()
        plt.title(title)
        plt.plot(x, trend_base[key][:len(x)], label='Base')
        plt.plot(x, trend_adv[key][:len(x)], label='Adversarial')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy' if 'acc' in key else 'Loss')
        plt.legend()
        filename = str(key + '_both' + ".png")
        print(filename)
        plt.savefig(filename)
        files.download(filename)
        plt.close()

#### Creates the graph with input/output nodes for training

In [0]:
def prepareTrainingModel(param):
    
    n_channels = param['CHANNELS']
    
    with tf.variable_scope('Baseline' if param['BASELINE'] else 'Adversarial'):
        # Graph
        x_train_ph = tf.placeholder(tf.float32)
        x_test_ph = tf.placeholder(tf.float32)
        y_train_ph = tf.placeholder(tf.float32)
        y_test_ph = tf.placeholder(tf.float32)

        train_op, train_loss, train_logit = CreateBaseModel(x_train_ph, y_train_ph, n_channels=n_channels, is_train=True) if param['BASELINE'] else CreateAdDModel(
            x_train_ph, y_train_ph, n_channels=n_channels, is_train=True)
        test_loss, test_logit = CreateTestModel(x_test_ph, y_test_ph, n_channels=n_channels, is_train=False)

        # Accuracy Train
        train_accuracy = Accuracy(train_logit, y_train_ph)

        # Accuracy Test
        test_accuracy = Accuracy(test_logit, y_test_ph)  
    
    nodes = {
        'x_train_ph': x_train_ph,
        'x_test_ph': x_test_ph,
        'y_train_ph': y_train_ph,
        'y_test_ph': y_test_ph,
        'train_op': train_op,
        'train_loss': train_loss,
        'train_logit': train_logit,
        'test_logit': test_logit,
        'test_loss': test_loss,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
    }
    
    return nodes
    

#### Can perform training incrementally i.e. you do not need to run all epochs at once

In [0]:
def doTraining(x_train, y_train, x_test, y_test, sess, nodes, param, trend, n_epochs=None):
    # Training setup

    batch_size = param['BATCH_SIZE']
    epochs = param['EPOCHS'] if n_epochs is None else n_epochs
    n_channels = x_train.shape[-1]

    STEPS = len(x_train) // batch_size if param['STEPS'] is None else param['STEPS']
    TEST_STEPS = len(x_test) // batch_size if param['STEPS'] is None else param['STEPS']
     
    # Training
    for epoch in range(epochs):
        # Train model
        acc_train, loss_train = 0, 0
        for i in range(STEPS):
            _, loss_, acc = sess.run([nodes['train_op'], nodes['train_loss'], nodes['train_accuracy']],
                                     feed_dict={nodes['x_train_ph']: x_train[batch_size * i: batch_size * (i + 1)],
                                                nodes['y_train_ph']: y_train[batch_size * i: batch_size * (i + 1)]})
            acc_train += acc
            loss_train += loss_

        trend['acc_train_trend'].append(acc_train / STEPS)
        trend['loss_train_trend'].append(loss_train / STEPS)

        # Test model
        acc_test, loss_test = 0, 0
        for i in range(TEST_STEPS):
            loss_t, acc_t = sess.run([nodes['test_loss'], nodes['test_accuracy']],
                                     feed_dict={nodes['x_test_ph']: x_test[batch_size * i: batch_size * (i + 1)],
                                                nodes['y_test_ph']: y_test[batch_size * i: batch_size * (i + 1)]})
            acc_test += acc_t
            loss_test += loss_t

        trend['acc_test_trend'].append(acc_test / TEST_STEPS)
        trend['loss_test_trend'].append(loss_test / TEST_STEPS)

        print('Epoch: {} || Train Loss: {}, Train Acc: {} || Test Loss: {}, Test Accuracy: {}'.format(epoch,
                                                                                                      loss_train / STEPS,
                                                                                                      acc_train / STEPS,
                                                                                                      loss_test / TEST_STEPS,
                                                                                                      acc_test / TEST_STEPS))
                  

#### Data I/O and preprocessing

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
# with tf.Session() as sess:
#     x_train = sess.run(tf.image.rgb_to_grayscale(x_train)) / 255
#     x_test = sess.run(tf.image.rgb_to_grayscale(x_test)) / 255
x_train, x_test = preprocess(x_train), preprocess(x_test)
    
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Adversarial Dropout training

In [0]:
param_adv = {
    'CHANNELS': 3,
    'BATCH_SIZE': 100,
    'EPOCHS': 50,
    'STEPS': None,
    'BASELINE': False
}

In [0]:
'''
Run this to delete default graph w/o having to restart notebook
'''
# tf.reset_default_graph()

In [0]:
nodes_adv = prepareTrainingModel(param_adv)

In [0]:
trend_adv = {
    'acc_train_trend': [],
    'loss_train_trend': [],
    'acc_test_trend': [],
    'loss_test_trend': []
}

In [0]:
sess_adv = tf.Session()
sess_adv.run(tf.global_variables_initializer())

In [106]:
doTraining(x_train, y_train, x_test, y_test, sess_adv, nodes_adv, param_adv, trend_adv, n_epochs=10)

Epoch: 0 || Train Loss: 1.620110316991806, Train Acc: 0.4129999985992909 || Test Loss: 1.2717525166273118, Test Accuracy: 0.5419999977946282
Epoch: 1 || Train Loss: 1.2598333572149276, Train Acc: 0.5546799973249436 || Test Loss: 1.0823384922742845, Test Accuracy: 0.621099999845028
Epoch: 2 || Train Loss: 1.1138328193426132, Train Acc: 0.610099999666214 || Test Loss: 0.9574610203504562, Test Accuracy: 0.6692000007629395
Epoch: 3 || Train Loss: 1.0106067848205567, Train Acc: 0.6480200009346009 || Test Loss: 0.8615555489063262, Test Accuracy: 0.700400002002716
Epoch: 4 || Train Loss: 0.9376442730426788, Train Acc: 0.6732400019168854 || Test Loss: 0.8257872021198273, Test Accuracy: 0.7164000022411346
Epoch: 5 || Train Loss: 0.8856502976417542, Train Acc: 0.6921800022125244 || Test Loss: 0.778647956252098, Test Accuracy: 0.7312000012397766
Epoch: 6 || Train Loss: 0.8425833998918534, Train Acc: 0.7071600006818771 || Test Loss: 0.7530433434247971, Test Accuracy: 0.7382000005245208
Epoch: 7 ||

In [108]:
doTraining(x_train, y_train, x_test, y_test, sess_adv, nodes_adv, param_adv, trend_adv, n_epochs=40)

Epoch: 0 || Train Loss: 0.7158123844265938, Train Acc: 0.7516199982166291 || Test Loss: 0.6704400649666786, Test Accuracy: 0.7714000010490417
Epoch: 1 || Train Loss: 0.6866864419579506, Train Acc: 0.7600599995851517 || Test Loss: 0.657285475730896, Test Accuracy: 0.7787999957799911
Epoch: 2 || Train Loss: 0.6669026043415069, Train Acc: 0.7685799983739853 || Test Loss: 0.6399628001451493, Test Accuracy: 0.784899999499321
Epoch: 3 || Train Loss: 0.6464293702244759, Train Acc: 0.7760199990272522 || Test Loss: 0.6306747069954872, Test Accuracy: 0.7871999973058701
Epoch: 4 || Train Loss: 0.6296521626114845, Train Acc: 0.7805399976968765 || Test Loss: 0.6239123818278313, Test Accuracy: 0.7900999993085861
Epoch: 5 || Train Loss: 0.6066504675745964, Train Acc: 0.7897199989557266 || Test Loss: 0.6041067487001419, Test Accuracy: 0.7974999976158142
Epoch: 6 || Train Loss: 0.5888659143447876, Train Acc: 0.7949599971771241 || Test Loss: 0.6097107145190239, Test Accuracy: 0.7976999998092651
Epoch: 7

Epoch: 36 || Train Loss: 0.30994190934300425, Train Acc: 0.8909200005531311 || Test Loss: 0.6472965139150619, Test Accuracy: 0.8221999990940094
Epoch: 37 || Train Loss: 0.30209072713553903, Train Acc: 0.893200000166893 || Test Loss: 0.6558462485671044, Test Accuracy: 0.8188999968767167
Epoch: 38 || Train Loss: 0.2970891007483005, Train Acc: 0.8960400013923645 || Test Loss: 0.6450983291864395, Test Accuracy: 0.8267999964952469
Epoch: 39 || Train Loss: 0.2956283106803894, Train Acc: 0.89634000146389 || Test Loss: 0.6504785293340682, Test Accuracy: 0.8233999997377396


In [117]:
doTraining(x_train, y_train, x_test, y_test, sess_adv, nodes_adv, param_adv, trend_adv, n_epochs=20)

Epoch: 0 || Train Loss: 0.28926089495420454, Train Acc: 0.898240000128746 || Test Loss: 0.6452942350506783, Test Accuracy: 0.8263999986648559
Epoch: 1 || Train Loss: 0.28459110695123674, Train Acc: 0.8992600003480912 || Test Loss: 0.6613492301106453, Test Accuracy: 0.8210999977588653
Epoch: 2 || Train Loss: 0.2812925135493278, Train Acc: 0.901300000667572 || Test Loss: 0.6619101724028588, Test Accuracy: 0.8210999977588653
Epoch: 3 || Train Loss: 0.276981046333909, Train Acc: 0.9019400004148483 || Test Loss: 0.64685526907444, Test Accuracy: 0.8249999988079071
Epoch: 4 || Train Loss: 0.27071644307672976, Train Acc: 0.9037400007247924 || Test Loss: 0.6614064973592758, Test Accuracy: 0.8246999984979629
Epoch: 5 || Train Loss: 0.2748752849698067, Train Acc: 0.9031199998855591 || Test Loss: 0.6581308054924011, Test Accuracy: 0.8269999974966049
Epoch: 6 || Train Loss: 0.27009704434871673, Train Acc: 0.90432000207901 || Test Loss: 0.6601120835542679, Test Accuracy: 0.8259999978542328
Epoch: 7 

In [0]:
with open('adv_trend.json', 'w') as fp:
    json.dump(trend_adv, fp)
files.download('adv_trend.json')

In [118]:
visualize(trend_adv, param_adv)

acc_train_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINEFalse.png
loss_train_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINEFalse.png
acc_test_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINEFalse.png
loss_test_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINEFalse.png


## Baseline Training

In [0]:
param_base = {
    'CHANNELS': 3,
    'BATCH_SIZE': 100,
    'EPOCHS': 50,
    'STEPS': None,
    'BASELINE': True
}

In [0]:
nodes_base = prepareTrainingModel(param_base)

In [0]:
trend_base = {
    'acc_train_trend': [],
    'loss_train_trend': [],
    'acc_test_trend': [],
    'loss_test_trend': []
}

In [0]:
sess_base = tf.Session()
sess_base.run(tf.global_variables_initializer())

In [124]:
doTraining(x_train, y_train, x_test, y_test, sess_base, nodes_base, param_base, trend_base, n_epochs=50)

Epoch: 0 || Train Loss: 1.6085399551391601, Train Acc: 0.41087999918311835 || Test Loss: 1.2658606195449829, Test Accuracy: 0.5464999970793724
Epoch: 1 || Train Loss: 1.2566290135383607, Train Acc: 0.5512999967336655 || Test Loss: 1.0664029359817504, Test Accuracy: 0.6236999964714051
Epoch: 2 || Train Loss: 1.1168474978208542, Train Acc: 0.6023199974298478 || Test Loss: 0.9503870350122452, Test Accuracy: 0.6666000032424927
Epoch: 3 || Train Loss: 1.0164762270450591, Train Acc: 0.6425400001406669 || Test Loss: 0.8758441573381424, Test Accuracy: 0.6956000036001205
Epoch: 4 || Train Loss: 0.9459592801332474, Train Acc: 0.6686400018930435 || Test Loss: 0.8247440886497498, Test Accuracy: 0.7176999992132187
Epoch: 5 || Train Loss: 0.8916789782047272, Train Acc: 0.6862600003480911 || Test Loss: 0.7887231945991516, Test Accuracy: 0.7306000006198883
Epoch: 6 || Train Loss: 0.841057443022728, Train Acc: 0.7055800009965897 || Test Loss: 0.7609857788681984, Test Accuracy: 0.7396000015735626
Epoch:

Epoch: 36 || Train Loss: 0.36685942676663397, Train Acc: 0.8688600002527237 || Test Loss: 0.6288772609829902, Test Accuracy: 0.8175999957323075
Epoch: 37 || Train Loss: 0.36233904108405113, Train Acc: 0.8706200001239777 || Test Loss: 0.6085212418437004, Test Accuracy: 0.8188000005483628
Epoch: 38 || Train Loss: 0.35268817690014836, Train Acc: 0.8732400002479553 || Test Loss: 0.6314712625741958, Test Accuracy: 0.8154999977350235
Epoch: 39 || Train Loss: 0.3503491114675999, Train Acc: 0.8766800001859665 || Test Loss: 0.6209696188569069, Test Accuracy: 0.8194999986886978
Epoch: 40 || Train Loss: 0.3353552449941635, Train Acc: 0.8809000005722046 || Test Loss: 0.6351635047793388, Test Accuracy: 0.819899998307228
Epoch: 41 || Train Loss: 0.33653587263822554, Train Acc: 0.8797399991750717 || Test Loss: 0.6309032583236694, Test Accuracy: 0.8195999962091446
Epoch: 42 || Train Loss: 0.32838118724524973, Train Acc: 0.8820000015497208 || Test Loss: 0.6379388278722763, Test Accuracy: 0.818899998664

In [0]:
with open('base_trend.json', 'w') as fp:
    json.dump(trend_base, fp)
files.download('base_trend.json')

In [125]:
visualize(trend_base, param_base)

acc_train_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINETrue.png
loss_train_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINETrue.png
acc_test_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINETrue.png
loss_test_trendCHANNELS3BATCH_SIZE100EPOCHS50STEPSNoneBASELINETrue.png


In [130]:
visualize_both(trend_base, trend_adv, param_base, param_adv)

acc_train_trend_both.png
loss_train_trend_both.png
acc_test_trend_both.png
loss_test_trend_both.png
